# Eat Safe, Love

## Part 1: Database and Jupyter Notebook Set Up

Import the data provided in the `establishments.json` file from your Terminal. Name the database `uk_food` and the collection `establishments`.

Within this markdown cell, copy the line of text you used to import the data from your Terminal. This way, future analysts will be able to repeat your process.

e.g.: Import the dataset with `YOUR IMPORT TEXT HERE`

In [1]:
!mongoimport --type json -d uk_food -c establishments --drop --jsonArray Resources/establishments.json

2023-09-16T09:27:51.667-0400	connected to: mongodb://localhost/
2023-09-16T09:27:51.688-0400	dropping: uk_food.establishments
2023-09-16T09:27:54.668-0400	[############............] uk_food.establishments	19.8MB/39.3MB (50.4%)
2023-09-16T09:27:57.372-0400	[########################] uk_food.establishments	39.3MB/39.3MB (100.0%)
2023-09-16T09:27:57.372-0400	39779 document(s) imported successfully. 0 document(s) failed to import.


In [2]:
# Import dependencies
from pymongo import MongoClient
from pprint import pprint

In [3]:
# Create an instance of MongoClient
mongo = MongoClient(port=27017)

In [4]:
# confirm that our new database was created
print(mongo.list_database_names())

['admin', 'config', 'epa', 'fruits_db', 'local', 'uk_food']


In [5]:
# assign the uk_food database to a variable name
db = mongo['uk_food']

In [6]:
# review the collections in our new database
print(db.list_collection_names())

['establishments']


In [7]:
# review a document in the establishments collection
print(db.establishments.find_one())

{'_id': ObjectId('6505ad57f3f274c12b5dbb19'), 'FHRSID': 254250, 'ChangesByServerID': 0, 'LocalAuthorityBusinessID': 'PI/000066174', 'BusinessName': 'The Pines Calyx', 'BusinessType': 'Other catering premises', 'BusinessTypeID': 7841, 'AddressLine1': 'The Pines Garden', 'AddressLine2': 'Beach Road', 'AddressLine3': 'St Margarets Bay', 'AddressLine4': 'Kent', 'PostCode': 'CT15 6DZ', 'Phone': '', 'RatingValue': '5', 'RatingKey': 'fhrs_5_en-gb', 'RatingDate': '2021-08-17T00:00:00', 'LocalAuthorityCode': '182', 'LocalAuthorityName': 'Dover', 'LocalAuthorityWebSite': 'http://www.dover.gov.uk/', 'LocalAuthorityEmailAddress': 'publicprotection@dover.gov.uk', 'scores': {'Hygiene': 0, 'Structural': 0, 'ConfidenceInManagement': 0}, 'SchemeType': 'FHRS', 'geocode': {'longitude': '1.383298', 'latitude': '51.148133'}, 'RightToReply': '', 'Distance': 4587.362402580997, 'NewRatingPending': False, 'meta': {'dataSource': None, 'extractDate': '0001-01-01T00:00:00', 'itemCount': 0, 'returncode': None, 'to

In [8]:
# assign the collection to a variable
establishments = db['establishments']

## Part 2: Update the Database

1. An exciting new halal restaurant just opened in Greenwich, but hasn't been rated yet. The magazine has asked you to include it in your analysis. Add the following restaurant "Penang Flavours" to the database.

In [9]:
# Create and Insert the new restaurant into the collection
new_restaurant={
    "BusinessName":"Penang Flavours",
    "BusinessType":"Restaurant/Cafe/Canteen",
    "BusinessTypeID":"",
    "AddressLine1":"Penang Flavours",
    "AddressLine2":"146A Plumstead Rd",
    "AddressLine3":"London",
    "AddressLine4":"",
    "PostCode":"SE18 7DY",
    "Phone":"",
    "LocalAuthorityCode":"511",
    "LocalAuthorityName":"Greenwich",
    "LocalAuthorityWebSite":"http://www.royalgreenwich.gov.uk",
    "LocalAuthorityEmailAddress":"health@royalgreenwich.gov.uk",
    "scores":{
        "Hygiene":"",
        "Structural":"",
        "ConfidenceInManagement":""
    },
    "SchemeType":"FHRS",
    "geocode":{
        "longitude":"0.08384000",
        "latitude":"51.49014200"
    },
    "RightToReply":"",
    "Distance":4623.9723280747176,
    "NewRatingPending":True
}
insert_result = establishments.insert_one(new_restaurant)

In [10]:
# Check that the new restaurant was inserted
print("Inserted new restaurant with ID:", insert_result.inserted_id)

Inserted new restaurant with ID: 6505ad5fdd7f65e3a414b9e9


2. Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the `BusinessTypeID` and `BusinessType` fields.

In [11]:
# Find the BusinessTypeID for "Restaurant/Cafe/Canteen" and return only the BusinessTypeID and BusinessType fields
query = {'BusinessType': 'Restaurant/Cafe/Canteen'}
fields = ['BusinessTypeID', 'BusinessType']
establishments.find_one(query, fields)

{'_id': ObjectId('6505ad57f3f274c12b5dbb1a'),
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1}

3. Update the new restaurant with the `BusinessTypeID` you found.

In [12]:
# Update the new restaurant with the correct BusinessTypeID
establishments.update_one(new_restaurant, {'$set':{'BusinessTypeID':1}})

In [13]:
# Confirm that the new restaurant was updated
establishments.find_one({'BusinessName':'Penang Flavours'})

{'_id': ObjectId('6505ad5fdd7f65e3a414b9e9'),
 'BusinessName': 'Penang Flavours',
 'BusinessType': 'Restaurant/Cafe/Canteen',
 'BusinessTypeID': 1,
 'AddressLine1': 'Penang Flavours',
 'AddressLine2': '146A Plumstead Rd',
 'AddressLine3': 'London',
 'AddressLine4': '',
 'PostCode': 'SE18 7DY',
 'Phone': '',
 'LocalAuthorityCode': '511',
 'LocalAuthorityName': 'Greenwich',
 'LocalAuthorityWebSite': 'http://www.royalgreenwich.gov.uk',
 'LocalAuthorityEmailAddress': 'health@royalgreenwich.gov.uk',
 'scores': {'Hygiene': '', 'Structural': '', 'ConfidenceInManagement': ''},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '0.08384000', 'latitude': '51.49014200'},
 'RightToReply': '',
 'Distance': 4623.972328074718,
 'NewRatingPending': True}

4. The magazine is not interested in any establishments in Dover, so check how many documents contain the Dover Local Authority. Then, remove any establishments within the Dover Local Authority from the database, and check the number of documents to ensure they were deleted.

In [14]:
# Find how many documents have LocalAuthorityName as "Dover"
establishments.count_documents({'LocalAuthorityName':'Dover'})

994

In [15]:
# Delete all documents where LocalAuthorityName is "Dover"
establishments.delete_many({'LocalAuthorityName':'Dover'})

In [16]:
# Check if any remaining documents include Dover
print(establishments.count_documents({'LocalAuthorityName':'Dover'}))

0


In [17]:
# Check that other documents remain with 'find_one'
establishments.find_one({})

{'_id': ObjectId('6505ad57f3f274c12b5dbdff'),
 'FHRSID': 647177,
 'ChangesByServerID': 0,
 'LocalAuthorityBusinessID': 'PI/000041489',
 'BusinessName': 'Wear Bay Bowls Club',
 'BusinessType': 'Pub/bar/nightclub',
 'BusinessTypeID': 7843,
 'AddressLine1': 'Wear Bay Road',
 'AddressLine2': 'Folkestone',
 'AddressLine3': 'Kent',
 'AddressLine4': '',
 'PostCode': 'CT19 6PY',
 'Phone': '',
 'RatingValue': '4',
 'RatingKey': 'fhrs_4_en-gb',
 'RatingDate': '2014-03-31T00:00:00',
 'LocalAuthorityCode': '188',
 'LocalAuthorityName': 'Folkestone and Hythe',
 'LocalAuthorityWebSite': 'http://www.folkestone-hythe.gov.uk',
 'LocalAuthorityEmailAddress': 'foodteam@folkestone-hythe.gov.uk',
 'scores': {'Hygiene': 5, 'Structural': 5, 'ConfidenceInManagement': 10},
 'SchemeType': 'FHRS',
 'geocode': {'longitude': '1.196408', 'latitude': '51.086058'},
 'RightToReply': '',
 'Distance': 4591.821311183521,
 'NewRatingPending': False,
 'meta': {'dataSource': None,
  'extractDate': '0001-01-01T00:00:00',
  '

5. Some of the number values are stored as strings, when they should be stored as numbers.

Use `update_many` to convert `latitude` and `longitude` to decimal numbers.

In [18]:
# Change the data type from String to Decimal for longitude and latitude
update = [{'$set': {
            "geocode.longitude": {'$toDouble': "$geocode.longitude"},
            "geocode.latitude": {'$toDouble': "$geocode.latitude"},
          }}]
establishments.update_many({}, update)

In [19]:
establishments.find_one({}, {"geocode":1})

{'_id': ObjectId('6505ad57f3f274c12b5dbdff'),
 'geocode': {'longitude': 1.196408, 'latitude': 51.086058}}

Use `update_many` to convert `RatingValue` to integer numbers.

In [20]:
# Set non 1-5 Rating Values to Null
non_ratings = ["AwaitingInspection", "Awaiting Inspection", "AwaitingPublication", "Pass", "Exempt"]
establishments.update_many({"RatingValue": {"$in": non_ratings}}, [ {'$set':{ "RatingValue" : None}} ])

In [21]:
# Change the data type from String to Integer for RatingValue
rating_query = {"RatingValue": {"$nin": non_ratings}}
rating_update = [{'$set': {
                "RatingValue": {'$toInt': "$RatingValue"},
                  }}]
establishments.update_many({}, rating_update)

In [22]:
# Check that the coordinates and rating value are now numbers
establishments.find_one({}, {"geocode":1, "RatingValue":1})

{'_id': ObjectId('6505ad57f3f274c12b5dbdff'),
 'RatingValue': 4,
 'geocode': {'longitude': 1.196408, 'latitude': 51.086058}}